# R.O.P Example: "ack" command wrapper using shellCommandROP

This is an example of how to "wrap" a unix command with shellCommandROP.

## TL;DR
Wrapper of "ack" command made with shellCommand.

## ROP and shellCommandROP: everything is there

In [1]:
//Export
public protocol DescriptiveError {
    func errorMessage()->String
}

public struct SimpleError: DescriptiveError {
    let msg: String // Immutable
    init(_ msg: String) { self.msg = msg }
    public func errorMessage() -> String { return msg }
}

public enum Result<OK> {
    case Ok(OK)
    case Error(DescriptiveError) // We always want an error that follows this protocol
    
    // Methods and Properties
    var value: OK? {
        get {
            switch self {
                case .Ok(let value): return value
                case .Error(_): return nil // Don't need the value
            }
        }
    }    
    var error: String? {
        get {
            switch self {
                case .Ok(_): return nil
                case .Error(let err): return err.errorMessage()
            }
        }
    }
    //MAP: spark of magic..
    func then<OUT>(_ f: (OK)->OUT ) -> Result<OUT> {
        switch self {
            case .Ok(let val): return Result<OUT>.Ok(f(val))
            case .Error(let err): return Result<OUT>.Error(err) // Forward error
        }
    }
    //FLAT MAT: the real magic of monad! 
    func then<OUT>(_ f: (OK)->Result<OUT> ) -> Result<OUT> {
        switch self {
            case .Ok(let val): return f(val)
            case .Error(let err): return Result<OUT>.Error(err) // Forward error
        }
    }
    //TAP: result snooping 
    func use(_ f: (OK)->() ) -> Result<OK> {
        switch self {
            case .Ok(let val): f(val)
            case .Error(let err): print("ERROR HAS HAPPENED: \(err.errorMessage())")
        }
        return self
    }
    //Convenience
    func isOk() -> Bool {
        switch self {
            case .Ok: return true
            case .Error: return false
        }
    }
}

//Convenience factory functions as static to quick find them with auto completition...
public func good<T>(_ value: T) -> Result<T> {
    return .Ok(value) // Thnx Chris ;-)
}
public func bad<T>(_ msg: String) -> Result<T> {
    return .Error(SimpleError(msg)) // Thnx Chris ;-)
}
//Create bad result from generic DescriptiveError
public func bad<T>(_ err: DescriptiveError) -> Result<T> {
    return Result<T>.Error(err)
}
//Forwards errors adapting type to destination...
public func bad<T,P>(_ parent: Result<P>) -> Result<T> {
    switch parent {
        case .Ok: return bad("WARNING: parent was not bad!") // good goes bad!! Warning!!
        case .Error(let val): return Result<T>.Error(val) // bad remains bad...
    }
}

//export
public struct ShellCommandError: DescriptiveError {
    let code: Int32 // Immutable
    let msg: String // Immutable
    init(_ code: Int32, _ msg: String) { 
        self.code = code
        self.msg = msg 
    }
    public func errorMessage() -> String { return "[\(code)]: \(msg)" }
}

//export
import Foundation

//export
@discardableResult
public func shellCommandROP(_ launchPath: String, _ arguments: [String] = []) -> Result<String>
{
    let task = Process()
    task.executableURL = URL(fileURLWithPath: launchPath)
    task.arguments = arguments

    let pipe = Pipe(); task.standardOutput = pipe
    let errPipe = Pipe(); task.standardError = errPipe
    do {
        try task.run()
        task.waitUntilExit()
        let data = pipe.fileHandleForReading.readDataToEndOfFile()
        let stdErr = errPipe.fileHandleForReading.readDataToEndOfFile()
        let text = String(data: data, encoding: String.Encoding.utf8) ?? ""
        let err = String(data: stdErr, encoding: String.Encoding.utf8) ?? ""
        if 0==task.terminationStatus {
            return good(text)
        } else {
            // Don't use the standard "bad" result, but keep termination status...
            return bad(ShellCommandError(task.terminationStatus, err))
        }
    } catch {
        return bad("ERROR: \(error).") // Incredible, to convert it I've just changed print to "bad"
    }
}

# Ack wrapper

### Support functions

In [2]:
//export 
func unixPwd() -> String {
    return shellCommandROP("/bin/pwd")    
        .then{pwd in pwd.replacingOccurrences(of: "\n", with: "")} // Clean pwd output form extra "\n"
        .value! // Should be safe: usually pwd is not null ;-)
}

In [3]:
print("'\(unixPwd())'")

'/notebooks/swift-rop/jupyter'


In [4]:
//export 

//act as doc in functional languages ;-) 
typealias FileName = String
typealias RowNumber = Int
typealias SampleText = String
typealias AckResult = (FileName, RowNumber, SampleText)

func processLine(_ src: String) -> AckResult {
    let tmp = src.split(separator: ":", maxSplits: 2, omittingEmptySubsequences: true)
    if tmp.count >= 3 {
        let file = String(tmp[0])
        let line = Int(tmp[1])! // Prtty sure it's a number
        let sample = String(tmp[2])
        return (file, line, sample)
    } else {
        return ("?", 0, "?")
    }
}

In [5]:
func test_processLine() {
    let sampleLine = "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift:72:/// A model learner, responsible for initializing and training a model on a given dataset."
    print("IN:", sampleLine)
    print("OUT:", processLine(sampleLine))
    print("OUT WITH ERRORS:", processLine("asdas dasd adas dasdas"))
}
test_processLine()

IN: /notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift:72:/// A model learner, responsible for initializing and training a model on a given dataset.
OUT: ("/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift", 72, "/// A model learner, responsible for initializing and training a model on a given dataset.")
OUT WITH ERRORS: ("?", 0, "?")


### ack implementation

In [6]:
//export
typealias AckResults = Result<[AckResult]>

func printAckResults(ackrRes: AckResults, printFullPath: Bool) {
    // Should check for errors...
    if let ackr=ackrRes.value {
        for row in ackr {
            let theFileName = (printFullPath) ? row.0 : (row.0 as NSString).lastPathComponent
            print(theFileName, "R:\(row.1)", row.2)
        }
    } else {
        print(">> No results found <<")
    }
}

func ack(word: String, path: String? = nil, caseInsensitive: Bool = true, show: Bool = true, printFullPath: Bool = false) -> AckResults {
    let thePath = path ?? unixPwd()
    let args = good([word, thePath])
        .then{args in caseInsensitive ? args + ["-i"] : args}
    let ret = shellCommandROP("/usr/bin/ack", args.value!)
        .then{str in str.components(separatedBy: "\n")}
        //.use{lines in print(lines)} // Debug
        .then{lines in lines.map(processLine)}    
        .then{rets in rets.filter{r in r.0 != "?"}} // Filtering out errors
        .then{rets in (rets.count>0) ? good(rets) : bad("No results found")} // No resuls is "bad" ;-) 
    if show {
        if ret.isOk() {
            printAckResults(ackrRes: ret, printFullPath: printFullPath)
        }
    }
    return ret
}

Let's make some tests...

In [7]:
ack(word: "float") // Search in current folder

rop-ack.ipynb R:320       "rop-ack.ipynb R:317     \"ack(word: \\\"float\\\") // Search in current folder\"\r\n",
rop-ack.ipynb R:321       "rop-ack.ipynb R:335       \"04_callbacks.swift R:77           // Constrain model input to Tensor<Float>, to work around\\r\\n\",\r\n",
rop-ack.ipynb R:322       "rop-ack.ipynb R:336       \"04_callbacks.swift R:79           Opt.Model.Input == Tensor<Float>\\r\\n\",\r\n",
rop-ack.ipynb R:323       "rop-ack.ipynb R:394        \"      - .2 : \\\"          // Constrain model input to Tensor<Float>, to work around\\\"\\n\",\r\n",
rop-ack.ipynb R:324       "rop-ack.ipynb R:398        \"      - .2 : \\\"          Opt.Model.Input == Tensor<Float>\\\"\\n\",\r\n",
rop-ack.ipynb R:325       "rop-ack.ipynb R:511       \"05b_early_stopping.swift R:25         // A learning rate schedule from step to float.\\r\\n\",\r\n",
rop-ack.ipynb R:326       "rop-ack.ipynb R:512       \"04_callbacks.swift R:123     /// The current epoch + iteration, float between 0.0 and e

▿ Result<Array<(String, Int, String)>>
  ▿ Ok : 43 elements
    ▿ 0 : 3 elements
      - .0 : "/notebooks/swift-rop/jupyter/rop-ack.ipynb"
      - .1 : 320
      - .2 : "      \"rop-ack.ipynb R:317     \\\"ack(word: \\\\\\\"float\\\\\\\") // Search in current folder\\\"\\r\\n\","
    ▿ 1 : 3 elements
      - .0 : "/notebooks/swift-rop/jupyter/rop-ack.ipynb"
      - .1 : 321
      - .2 : "      \"rop-ack.ipynb R:335       \\\"04_callbacks.swift R:77           // Constrain model input to Tensor<Float>, to work around\\\\r\\\\n\\\",\\r\\n\","
    ▿ 2 : 3 elements
      - .0 : "/notebooks/swift-rop/jupyter/rop-ack.ipynb"
      - .1 : 322
      - .2 : "      \"rop-ack.ipynb R:336       \\\"04_callbacks.swift R:79           Opt.Model.Input == Tensor<Float>\\\\r\\\\n\\\",\\r\\n\","
    ▿ 3 : 3 elements
      - .0 : "/notebooks/swift-rop/jupyter/rop-ack.ipynb"
      - .1 : 323
      - .2 : "      \"rop-ack.ipynb R:394        \\\"      - .2 : \\\\\\\"          // Constrain model input to Tensor

In [8]:
ack(word: "model", path: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block") // Search in folder

08_data_block.swift R:251 public struct CNNModel: Layer {
06_cuda.swift R:21 public struct CnnModel: Layer {
07_batchnorm.swift R:118 public struct CnnModelBN: Layer {
04_callbacks.swift R:10 public struct BasicModel: Layer {
04_callbacks.swift R:72 /// A model learner, responsible for initializing and training a model on a given dataset.
04_callbacks.swift R:76           Opt.Model: Layer,
04_callbacks.swift R:77           // Constrain model input to Tensor<Float>, to work around
04_callbacks.swift R:79           Opt.Model.Input == Tensor<Float>
04_callbacks.swift R:82     public typealias Model = Opt.Model
04_callbacks.swift R:83     public typealias Input = Model.Input
04_callbacks.swift R:84     public typealias Output = Model.Output
04_callbacks.swift R:88     public typealias Variables = Model.AllDifferentiableVariables
04_callbacks.swift R:94         public typealias F = @differentiable (Model.Output, @nondiff Label) -> Loss
04_callbacks.swift R:99     /// The dataset on which th

▿ Result<Array<(String, Int, String)>>
  ▿ Ok : 31 elements
    ▿ 0 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/08_data_block.swift"
      - .1 : 251
      - .2 : "public struct CNNModel: Layer {"
    ▿ 1 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/06_cuda.swift"
      - .1 : 21
      - .2 : "public struct CnnModel: Layer {"
    ▿ 2 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/07_batchnorm.swift"
      - .1 : 118
      - .2 : "public struct CnnModelBN: Layer {"
    ▿ 3 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift"
      - .1 : 10
      - .2 : "public struct BasicModel: Layer {"
    ▿ 4 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/Fa

In [9]:
// Search case sensitive
ack(word: "float", path: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block", caseInsensitive: false)

05b_early_stopping.swift R:25         // A learning rate schedule from step to float.
04_callbacks.swift R:123     /// The current epoch + iteration, float between 0.0 and epochCount
05_anneal.swift R:150         // A learning rate schedule from step to float.


▿ Result<Array<(String, Int, String)>>
  ▿ Ok : 3 elements
    ▿ 0 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05b_early_stopping.swift"
      - .1 : 25
      - .2 : "        // A learning rate schedule from step to float."
    ▿ 1 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift"
      - .1 : 123
      - .2 : "    /// The current epoch + iteration, float between 0.0 and epochCount"
    ▿ 2 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05_anneal.swift"
      - .1 : 150
      - .2 : "        // A learning rate schedule from step to float."


In [10]:
// Full path on result
ack(word: "float", path: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block", caseInsensitive: false, printFullPath: true)

/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05b_early_stopping.swift R:25         // A learning rate schedule from step to float.
/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift R:123     /// The current epoch + iteration, float between 0.0 and epochCount
/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05_anneal.swift R:150         // A learning rate schedule from step to float.


▿ Result<Array<(String, Int, String)>>
  ▿ Ok : 3 elements
    ▿ 0 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05b_early_stopping.swift"
      - .1 : 25
      - .2 : "        // A learning rate schedule from step to float."
    ▿ 1 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/04_callbacks.swift"
      - .1 : 123
      - .2 : "    /// The current epoch + iteration, float between 0.0 and epochCount"
    ▿ 2 : 3 elements
      - .0 : "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block/Sources/FastaiNotebook_08_data_block/05_anneal.swift"
      - .1 : 150
      - .2 : "        // A learning rate schedule from step to float."


In [11]:
ack(word: "xxxx" + String(Int.random(in: 0 ..< 10))) // No result

▿ Result<Array<(String, Int, String)>>
  ▿ Error : ShellCommandError
    - code : 1
    - msg : ""


In [12]:
ack(word: "xxxx", path: "/asd/asd") // Wrong folder

▿ Result<Array<(String, Int, String)>>
  ▿ Error : ShellCommandError
    - code : 1
    - msg : "ack: /asd/asd: No such file or directory\n"


In [13]:
//No output
let ret = ack(word: "model", path: "/notebooks/fastai_docs/dev_swift/FastaiNotebook_08_data_block", show: false)
print("Was last result successfull??", ret.isOk())

Was last result successfull?? true
